# 4-6 OpenPose の学習

## 学習の注意点
損失関数の定義と訓練用の Dataset に検証用データを使用する点に注意する．

## DataLoader と Network の作成
検証用の DataLoader を作成しないので，訓練用と検証用の DataLoader をまとめた辞書型変数 dataloaders_dict の val 側は None に設定する．
なおミニバッチサイズは GPU メモリに載る範囲で最大に近い 32 と設定している．

In [1]:
import torch
import torch.utils.data as data
from utils.dataloader import make_datapath_list, DataTransform, COCOkeypointsDataset

# COCO データセットのファイルパスリストを作成
train_img_list, train_mask_list, val_img_list, val_mask_list, train_meta_list, val_meta_list = make_datapath_list(rootpath="./data/")

# Dataset の作成
# train を val_list で作成しているため注意
train_dataset = COCOkeypointsDataset(val_img_list, val_mask_list, val_meta_list, phase="train", transform=DataTransform())

# 今回は簡易な学習とし検証データは作成しない
# val_dataset = CocokeypointsDataset(val_img_list, val_mask_list, val_meta_list, phase="val", transform=DataTransform())

# DataLoader 作成
batch_size = 8
train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_dataloader = data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# 辞書型変数にまとめる
dataloaders_dict = {"train": train_dataloader, "val": None}
# dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

In [2]:
from utils.openpose_net import OpenPoseNet

net = OpenPoseNet()

## 損失関数の定義
OpenPose の損失関数は heatmaps と PAFs のそれぞれについて正解アノテーションデータとの回帰の誤差になる．
つまり，各ピクセルの値が教師データの値とどの程度近い値になるのか，ピクセルごとの値を回帰する．
そのため，損失関数はセマンティックセグメンテーションとは異なるものとなる．  
OpenPose の損失関数は回帰問題で一般に使われる平均二乗誤差とし，6つある各 Stage ごとに損失を計算する．
ネットワークモデルの全体の誤差としては，各 Stage の全ての誤差を足し合わせたものとする．  
人物が写っているが姿勢のアノテーションがない部分については損失を計算しない．
そこで教師データのアノテーションと各 Stage で推定した内容のどちらにも mask（無視する部分の値が0，そうでない部分の値は1）をかけ算する．

In [3]:
import random
import math
import time
import pandas as pd
import numpy as np
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# 損失関数の設定
class OpenPoseLoss(nn.Module):
    ''' OpenPose の損失関数'''
    
    def __init__(self):
        super(OpenPoseLoss, self).__init__()
        
    
    def forward(self, saved_for_loss, heatmap_target, heat_mask, paf_target, paf_mask):
        """
        損失関数の計算
        
        Parameters
        ----------
        saved_for_loss : OpenPoseNet の出力 ( リスト )
        heatmap_target : [num_batch, 19, 46, 46]
            正解の部位のアノテーション情報
        heatmap_mask : [num_batch, 19, 46, 46]
            heatmap 画像の mask
        paf_target : [num_batch, 38, 46, 46]
            正解の PAF のアノテーション情報
        paf_mask : [num_batch, 38, 46, 46]
            PAF 画像の mask
            
        Returns
        -------
        loss : テンソル
            損失の値
        """
        
        total_loss = 0
        # ステージごとに計算
        for j in range(6):
            # PAFs と heatmaps においてマスクされている部分(paf_mask=0 など)は無視させる
            # PAFs
            pred1 = saved_for_loss[2 * j] * paf_mask
            gt1 = paf_target.float() * paf_mask
            
            # heatmaps
            pred2 = saved_for_loss[2 * j + 1] * heat_mask
            gt2 = heatmap_target.float() * heat_mask
            
            total_loss += F.mse_loss(pred1, gt1, reduction="mean") + F.mse_loss(pred2, gt2, reduction="mean")
            
        return total_loss
    
criterion = OpenPoseLoss()

## 学習の実行
本来であれば epoch ごとに徐々に学習率が小さくなるように設定するが，ここでは雰囲気の確認に留めるため簡単に設定する．

In [4]:
optimizer = optim.SGD(net.parameters(), lr=1e-2, momentum=0.9, weight_decay=0.0001)

ここでは検証フェーズを省略する．
また，これまでとは異なり ```if imges.size()[0]== 1:``` という部分でミニバッチのサイズが 1 になっていないかをチェックしている．
これは，PyTorch でバッチノーマライゼーションを行うときにミニバッチサイズが1だとエラーになるためである．

In [5]:
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    # GPU が使えることを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    
    # ネットワークを GPU へ
    net.to(device)
    
    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True
    
    # 画像の枚数
    num_train_imgs = len(dataloaders_dict["train"].dataset)
    batch_size = dataloaders_dict["train"].batch_size

    # イテレーションカウンタをセット
    iteration = 1
    
    # 学習ループ
    for epoch in range(num_epochs):
        # 開始時刻を保存
        t_epoch_start = time.time()
        t_iter_start = time.time()
        epoch_train_loss = 0.0 # epoch の損失和
        epoch_val_loss = 0.0 # epoch の損失和
        
        print('-------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')
        
        # epoch ごとの訓練と検証のループ
        for phase in ["train", "val"]:
            if phase == "train":
                net.train()
                optimizer.zero_grad()
                print(' (train) ')
                
            # 今回は検証はスキップ
            else:
                continue
                # net.eval()
                # print('-------------')
                # print('(val)')
                
            # データローダから minibatch ずつ取り出すループ
            for images, heatmap_target, heat_mask, paf_target, paf_mask in dataloaders_dict[phase]:
                # ミニバッチサイズが１だとエラーになる
                if images.size()[0] == 1:
                    continue
                
                # GPU が使えれば GPU を使う
                images = images.to(device)
                heatmap_target = heatmap_target.to(device)
                heat_mask = heat_mask.to(device)
                paf_target = paf_target.to(device)
                paf_mask = paf_mask.to(device)
                
                # optimizer を初期化
                optimizer.zero_grad()
                
                # foeward の計算
                with torch.set_grad_enabled(phase == "train"):
                    # (out6_1, out6_2) は使わないので _ で代替
                    _, saved_for_loss = net(images)
                    loss = criterion(saved_for_loss, heatmap_target, heat_mask, paf_target, paf_mask)
                    
                    # 訓練時はバックプロパゲーション
                    if phase == "train":
                        loss.backward()
                        optimizer.step()
                        
                        if (iteration % 10 == 0): # 10iter に 1 度、loss を表示
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print( ' イテレーション {} || Loss: {:.4f} || 10iter:{:.4f} sec.'.format(iteration, loss.item()/batch_size, duration))
                            t_iter_start = time.time()
                        
                        epoch_train_loss += loss.item()
                        iteration += 1
                    # 検証時
                    # else:
                        #epoch_val_loss += loss.item()
                        
                    # epochのphaseごとのlossと正解率
                    t_epoch_finish = time.time()
                    print('-------------')
                    print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch_VAL_Loss:{:.4f}'.format(epoch+1, epoch_train_loss/num_train_imgs, 0))
                    print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
                    t_epoch_start = time.time()

                    # 最後のネットワークを保存する
                    torch.save(net.state_dict(), 'weights/openpose_net_' + str(epoch+1) + '.pth')

In [6]:
# 学習・検証を実行する
num_epochs = 2
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

使用デバイス： cuda:0
-------------
Epoch 1/2
-------------
 (train) 


RuntimeError: CUDA out of memory. Tried to allocate 12.00 MiB (GPU 0; 3.79 GiB total capacity; 2.14 GiB already allocated; 19.31 MiB free; 17.30 MiB cached)